In [4]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
# from d2l import torch as d2l
import torch.optim as optim
import numpy as np

In [7]:
class RNNscratch(nn.Module):
    def __init__(self , input_size , hidden_size , output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.tanh = nn.Tanh()
        #Define the layers
        self.input_to_hidden = nn.Linear(input_size, hidden_size)
        self.hidden_to_hidden = nn.Linear(hidden_size, hidden_size)
        self.hidden_to_output = nn.Linear(hidden_size, output_size)
    def forward(self , x , hidden =None):
        batch_size , seq_len , _ = x.size()
        #initialize the hidden layer if not provided
        if hidden is None:
            hidden = torch.zeros(batch_size , self.hidden_size)
        outputs = []
        for t in range(seq_len):
            # get the input at the time step t
            x_t= x[:,t,:]
            #update the hidden state
            hidden = self.tanh(
                self.input_to_hidden(x_t) + self.hidden_to_hidden(hidden)
            )
            # get the output
            output = self.hidden_to_output(hidden)
            outputs.append(output)
        outputs = torch.stack(outputs, dim=1)
        return outputs, hidden

## Train function

In [8]:
# Training function - no manual backward pass needed!
def train_rnn(model, X_train, y_train, epochs=100, learning_rate=0.01):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    losses = []
    
    for epoch in range(epochs):
        total_loss = 0
        
        for i in range(len(X_train)):
            # Convert to tensors
            inputs = torch.FloatTensor(X_train[i]).unsqueeze(0)  # Add batch dimension
            targets = torch.FloatTensor(y_train[i]).unsqueeze(0)
            
            # Forward pass
            outputs, _ = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass - PyTorch handles this automatically!
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(X_train)
        losses.append(avg_loss)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")
    
    return losses


In [10]:
# Generate data (same as before)
def generate_sine_data(seq_length=20, num_sequences=100):
    X = []
    y = []
    
    for _ in range(num_sequences):
        start = np.random.uniform(0, 2*np.pi)
        x_seq = []
        y_seq = []
        
        for i in range(seq_length):
            x_val = start + i * 0.1
            x_seq.append([np.sin(x_val)])
            y_seq.append([np.sin(x_val + 0.1)])
        
        X.append(x_seq)
        y.append(y_seq)
    
    return X, y

# Create and train the model
X_train, y_train = generate_sine_data()

# Using custom RNN
model = RNNscratch(input_size=1, hidden_size=10, output_size=1)
losses = train_rnn(model, X_train, y_train, epochs=100, learning_rate=0.01)

# Or using built-in RNN
# model = SimpleRNN(input_size=1, hidden_size=10, output_size=1)

# Test the model
test_input = torch.FloatTensor([[np.sin(i * 0.1)] for i in range(20)]).unsqueeze(0)
model.eval()
with torch.no_grad():
    predictions, _ = model(test_input)
    print("Predictions:", predictions.squeeze().numpy())


Epoch 0, Loss: 0.1047
Epoch 10, Loss: 0.0025
Epoch 20, Loss: 0.0015
Epoch 30, Loss: 0.0012
Epoch 40, Loss: 0.0009
Epoch 50, Loss: 0.0009
Epoch 60, Loss: 0.0009
Epoch 70, Loss: 0.0008
Epoch 80, Loss: 0.0007
Epoch 90, Loss: 0.0007
Predictions: [0.04387655 0.22395891 0.26156637 0.39767507 0.4844117  0.56214964
 0.65358615 0.7198126  0.7864299  0.843774   0.89287955 0.93449736
 0.9665482  0.98992527 1.0040014  1.0084616  1.0031332  0.9877324
 0.96217585 0.926478  ]


In [11]:
def plot_predictions(model, test_input, test_target):
    model.eval()
    with torch.no_grad():
        predictions, _ = model(test_input)
        
    # Convert to numpy for plotting
    real_values = test_target.squeeze().numpy()
    pred_values = predictions.squeeze().numpy()
    
    plt.figure(figsize=(10, 5))
    plt.plot(real_values, label='Real', color='blue')
    plt.plot(pred_values, label='Predicted', color='orange', linestyle='--')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.title('Predicted vs Real Values')
    plt.legend()
    plt.grid(True)
    plt.show()
plot_predictions(model , test_input ,test_input )

